In [1]:
import pandas as pd
import numpy as np
import json
import pickle
from scipy import stats

In [2]:
with open('train_label.json') as f:
    expert_annotations = json.load(f)
with open('test.json') as f:
    test_labels = json.load(f)
with open('train.json') as f:
    train_labels = json.load(f)

In [3]:
with open('train_label.json') as f:
    expert_annotations2 = json.load(f)

In [4]:
np.random.seed(0)
our_test_examples = np.random.choice(expert_annotations, 750, replace=False)
exclude = [x['name'] for x in our_test_examples]

np.random.seed(0)
new_train_examples = np.random.choice(test_labels, 750, replace=False)
new_include= [x['name'] for x in new_train_examples]

pd.DataFrame({'exclude_from_train':exclude, 'include_in_train':new_include}).to_csv('test_examples.csv')

In [5]:
annotations = pd.DataFrame(expert_annotations)

In [6]:
# create combined dataframe of expert + model predictions
n_experts = 3
annotator_id_cols = ['label_A','label_B','label_C']

m = pd.read_csv("model_predictions.csv", index_col=0)
m['image'] = m['image'].apply(lambda x: x[23:])

df = annotations.merge(m, left_on='name', right_on='image')
df = df.drop('image', axis=1)
df['Y_H'] = df.apply(lambda x: [x[c] for c in annotator_id_cols], axis=1)
df['consensus'] = df['Y_H'].apply(lambda x: stats.mode(x, keepdims=True)[0][0])

df.columns = ['label', 'name', 'expert2', 'expert1', 'expert3', 'pred', 'model_p0', 'model_p1',
       'model_p2', 'model_p3', 'Y_H', 'consensus']
df = df[['name', 'expert1', 'expert2', 'expert3', 'model_p0', 'model_p1',
       'model_p2', 'model_p3', 'Y_H', 'consensus']]

df = df.sample(frac=1, random_state=0).reset_index(drop=True)

df.to_csv('data_clean.csv', index=None)
df.head()

,name,expert1,expert2,expert3,model_p0,model_p1,model_p2,model_p3,Y_H,consensus
0,train/539668_1-IMG013x030-2.JPG,2,2,2,0.047320,0.001019,0.950691,0.000970,"[2, 2, 2]",2
1,train/535990_2-IMG010x011-0.JPG,0,0,0,0.787148,0.211914,0.000021,0.000917,"[0, 0, 0]",0
2,train/538377-IMG013x006-0.JPG,0,0,0,0.992868,0.006686,0.000005,0.000441,"[0, 0, 0]",0
3,train/535943_1-IMG003x012-0.JPG,0,0,0,0.967250,0.003528,0.022085,0.007137,"[0, 0, 0]",0
4,train/540293_2-IMG021x014-2.JPG,2,2,2,0.002436,0.000017,0.997191,0.000356,"[2, 2, 2]",2
